# Virtual Sensing - Linear Regression

In [18]:
# libraries
# import datetime
# from datetime import date, timedelta
# from os import path
# import statistics
# import boto3
# import s3fs
# from fastparquet import ParquetFile
import pandas as pd
import numpy as np 
import warnings
from copy import deepcopy
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import sys
sys.path.append("../HistoricalData/")
from getData import get_data

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 500)

In [5]:
# constants
from getData import get_data
UP_LEFT = (38.008050, -122.536985)    
UP_RIGHT = (38.008050, -122.186437)   
DOWN_RIGHT = (37.701933, -122.186437) 
DOWN_LEFT = (37.701933, -122.536985)  
START_DATE = '2019/09/01' 
END_DATE = '2019/09/02'   
START_HOUR = '0'        
END_HOUR = '24'   

In [6]:
# load data into dataframe
data_df = get_data(UP_LEFT, UP_RIGHT, DOWN_RIGHT, DOWN_LEFT, START_DATE, END_DATE, START_HOUR, END_HOUR, 'Monthly')

In [25]:
data_df['created'] =  pd.to_datetime(data_df['created'], format='%Y%m%d%H%M')
data_df.head()
# data_df.columns

,0_3um,0_5um,1_0um,2_5um,5_0um,10_0um,pm1_0,pm10_0,created,pm1_0_atm,pm2_5_atm,pm10_0_atm,uptime,rssi,temperature,humidity,pm2_5_cf_1,device_loc_typ,is_owner,sensor_id,sensor_name,parent_id,lat,lon,thingspeak_primary_id,thingspeak_primary_id_read_key,thingspeak_secondary_id,thingspeak_secondary_id_read_key,a_h,high_reading_flag,hidden,city,county,zipcode,created_at,year,month,day,hour,minute,wban_number,call_sign,call_sign2,interval,call_sign3,zulu_time,report_modifier,wind_data,wind_direction,wind_speed,gusts,gust_speed,variable_winds,variable_wind_info,sys_maint_reqd,agency_name,aqi,category,epa_pm25_unit,epa_pm25_value,full_aqs_code,intl_aqs_code,raw_concentration,site_name,wkday,daytype,timeofday,wind_compass
0,563.51,162.10,33.51,5.16,1.25,0.00,1.96,4.93,2019-09-01 00:00:00,1.96,3.99,4.93,59078.0,-74.0,81.0,49.0,3.99,outside,0,16939,#SAFQ11,nan,37.72244,-122.439302,592280,YE7QMQYRJVM7WIH3,592281,13R50S7WTRLK96SD,None,nan,None,San Francisco,San Francisco County,94112,2019/09/01T00:00,2019,09,01,00,00,23234,KSFO,SFO,5-MIN,KSFO,010800Z,,1.0,280,7.0,0.0,NaN,0.0,,0.0,San Francisco Bay Area AQMD,21.000000,1.0,UG/M3,5.000000,60750005.0,8.400608e+11,4.000000,San Francisco,0,Weekend,night,West
1,519.13,149.87,35.50,6.77,1.54,0.43,1.91,5.31,2019-09-01 00:10:00,1.91,4.14,5.31,59088.0,-77.0,79.0,50.0,4.14,outside,0,16939,#SAFQ11,nan,37.72244,-122.439302,592280,YE7QMQYRJVM7WIH3,592281,13R50S7WTRLK96SD,None,nan,None,San Francisco,San Francisco County,94112,2019/09/01T00:10,2019,09,01,00,10,23234,KSFO,SFO,5-MIN,KSFO,010810Z,,1.0,290,7.0,0.0,NaN,0.0,,0.0,San Francisco Bay Area AQMD,20.333333,1.0,UG/M3,4.833333,60750005.0,8.400608e+11,3.833333,San Francisco,0,Weekend,night,ERROR
2,510.72,152.80,28.90,3.13,0.62,0.00,1.80,4.10,2019-09-01 00:20:00,1.80,3.59,4.10,59098.0,-73.0,79.0,50.0,3.59,outside,0,16939,#SAFQ11,nan,37.72244,-122.439302,592280,YE7QMQYRJVM7WIH3,592281,13R50S7WTRLK96SD,None,nan,None,San Francisco,San Francisco County,94112,2019/09/01T00:20,2019,09,01,00,20,23234,KSFO,SFO,5-MIN,KSFO,010820Z,,1.0,280,8.0,0.0,NaN,0.0,,0.0,San Francisco Bay Area AQMD,19.666667,1.0,UG/M3,4.666667,60750005.0,8.400608e+11,3.666667,San Francisco,0,Weekend,night,West
3,507.48,149.75,30.55,6.61,3.36,1.30,1.52,5.52,2019-09-01 00:30:00,1.52,3.35,5.52,59108.0,-75.0,79.0,50.0,3.35,outside,0,16939,#SAFQ11,nan,37.72244,-122.439302,592280,YE7QMQYRJVM7WIH3,592281,13R50S7WTRLK96SD,None,nan,None,San Francisco,San Francisco County,94112,2019/09/01T00:30,2019,09,01,00,30,23234,KSFO,SFO,5-MIN,KSFO,010830Z,,1.0,280,8.0,0.0,NaN,0.0,,0.0,San Francisco Bay Area AQMD,19.000000,1.0,UG/M3,4.500000,60750005.0,8.400608e+11,3.500000,San Francisco,0,Weekend,night,West
4,478.95,141.67,33.39,2.97,0.00,0.00,1.88,3.85,2019-09-01 00:40:00,1.88,3.85,3.85,59118.0,-71.0,78.0,51.0,3.85,outside,0,16939,#SAFQ11,nan,37.72244,-122.439302,592280,YE7QMQYRJVM7WIH3,592281,13R50S7WTRLK96SD,None,nan,None,San Francisco,San Francisco County,94112,2019/09/01T00:40,2019,09,01,00,40,23234,KSFO,SFO,5-MIN,KSFO,010840Z,,1.0,280,8.0,0.0,NaN,0.0,,0.0,San Francisco Bay Area AQMD,18.333333,1.0,UG/M3,4.333333,60750005.0,8.400608e+11,3.333333,San Francisco,0,Weekend,night,West


In [26]:
# get rid of rows that have no wind data? uncomment if yes.
# data_df = data_df[data_df.wind_data == 1.0]

In [8]:
data_df.shape

(726292, 68)

In [10]:
# winnow down the features
columns_to_keep = ['lat', 'lon'] 
X_data_df = data_df[columns_to_keep]
y_data_df = data_df['2_5um']

In [11]:
# fix datatyping
X_data_df.dtypes

created      int64
lat        float64
lon        float64
dtype: object

In [12]:
for col in X_data_df.columns:
    testy = X_data_df[col]
    print(col, testy.isna().sum())

created 0
lat 0
lon 0


In [75]:
# naively dump any rows with missing data
# X_data_df.dropna(inplace = True)

In [13]:
# X_data_df.shape

(726292, 3)

In [16]:
# split data into train-dev-test
X_train_and_dev, X_test, y_train_and_dev, y_test = train_test_split(X_data_df, y_data_df, test_size=0.20, random_state=42)
X_train, X_dev, y_train, y_dev = train_test_split(X_train_and_dev, y_train_and_dev, test_size=0.125, random_state=42)
print(X_train.shape)
print(X_dev.shape)
print(X_test.shape)

(508403, 3)
(72630, 3)
(145259, 3)


In [20]:
# fit the data
regr = KNeighborsRegressor()
regr.fit(X_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                    weights='uniform')

In [22]:
# make predictions
y_pred = regr.predict(X_dev)

In [ ]:
# calculate accuracy